<a href="https://colab.research.google.com/github/am-khan/affect/blob/master/BOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

from sklearn import svm
from sklearn import preprocessing

import numpy as np
import pandas as pd
import re
import csv
import os

## Load Data

In [12]:
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [] 

# FOR POCCA
all_poems_file = '../PAES/poems/'
for file in tqdm(os.listdir(all_poems_file)):
    if file[-4:] == '.txt':
        path = os.path.join(all_poems_file, file)
        corpus.append(open(path).read())

# # FOR FRIENDS
# for file in ['data/Friends/train.csv', 'data/Friends/dev.csv']:
#     with open(file) as f:
#         df = pd.read_csv(f)
#         corpus += list(df['content'])

len(corpus)

12818

In [13]:
# FOR POCCA
DATA_DIR = os.path.abspath('POCCA/')

# FOR FRIENDS
# DATA_DIR = os.path.abspath('data/Friends')
def get_datasets(data_dir: str):
    datasets = {}
    for partition in ["train","test"]:
        df = pd.read_csv(os.path.join(DATA_DIR, f'{partition}.csv'))
        datasets[partition] = df
    return datasets   

datasets = get_datasets(DATA_DIR)
datasets['train']

,title,interest,amusement,pride,joy,pleasure,contentment,love,admiration,relief,...,fear,disgust,contempt,hate,anger,valence,arousal,agreement,authors,content
0,100 Bells,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.13,0.43,0.17,0.13,0.43,-0.10,0.06,0.480000,Tarfia Faizullah,My sister died. He raped me. They beat me. I f...
1,A Symmetry,0.23,0.27,0.0,0.03,0.07,0.10,0.07,0.00,0.00,...,0.10,0.10,0.00,0.00,0.00,0.01,0.03,0.130000,Ari Banias,The magnolia before it blooms stands\nbare as ...
2,After Noise,0.28,0.02,0.0,0.00,0.00,0.10,0.00,0.10,0.00,...,0.00,0.00,0.00,0.00,0.00,-0.01,-0.04,0.240000,Sueyeun Juliette Lee,and who are you now \n in this dif...
3,April,0.15,0.20,0.0,0.35,0.15,0.15,0.00,0.00,0.10,...,0.00,0.20,0.30,0.00,0.00,0.03,0.06,-0.130000,Alicia Ostriker,The optimists among us\ntaking heart because i...
4,Artificial Death,0.00,0.00,0.0,0.00,0.03,0.10,0.00,0.23,0.13,...,0.10,0.00,0.00,0.00,0.00,0.00,-0.06,0.380000,Roberto Harrison,back 90 suns to a run through the ferns\nI rem...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,The House Gift,0.50,0.30,0.1,0.20,0.10,0.10,0.20,0.20,0.20,...,0.30,0.20,0.10,0.10,0.20,0.00,0.00,0.281128,Joanie Mackowski,"Egg-white house, old\nache in the rafters,\nsm..."
267,The Icehouse in Summer,0.60,0.30,0.2,0.40,0.50,0.40,0.40,0.40,0.50,...,0.30,0.30,0.30,0.20,0.40,0.10,-0.00,0.465487,Howard Nemerov,"A door sunk in a hillside, with a bolt\nthick ..."
268,The Journey,0.50,0.40,0.3,0.40,0.30,0.30,0.10,0.20,0.20,...,0.70,0.50,0.50,0.40,0.30,-0.20,-0.00,-0.052456,Yvor Winters,And then the dark fell and 'there has never' ...
269,The Lake,0.50,0.60,0.4,0.30,0.30,0.30,0.20,0.40,0.40,...,0.20,0.20,0.40,0.30,0.30,0.00,0.00,0.046172,Sophie Cabot Black,"Day and night, the lake dreams of sky.\nA priv..."


## TFIDF

In [14]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(f'vocab size: {X.toarray().shape[1]}')

vocab size: 72123


In [6]:
print(vectorizer.transform(['hello this is a document', 'this is another']).toarray().shape)

(2, 72123)


In [15]:
import os
import pandas as pd
import re
from tqdm.notebook import tqdm

poems = datasets['train']['content']

# FOR POCCA
def get_sadness_values(df):
    array = df['sadness'].to_numpy()
    return np.where(array>0.3, 1, 0)

def get_valence_values(df):
    array = df['arousal'].to_numpy()
    return np.where(array>0, 1, 0)

def get_affect_regression(df):
    array = np.asarray([df['arousal'],df['valence']])
    return array.T

# FOR FRIENDS
def get_label(df):
    return np.asarray(df['label'])


def get_agreement(df):
    return np.asarray(df['agreement'])

## Classifiers

In [52]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import f1_score, accuracy_score
labels = get_valence_values

X_train = vectorizer.transform(datasets['train']['content'])
Y_train = labels(datasets['train'])

X_test = vectorizer.transform(datasets['test']['content'])
Y_test = labels(datasets['test'])

print(X_train.shape, Y_train.shape)
print(X_test.shape)

clf = BernoulliNB()
clf.fit(X_train, Y_train)

Y_predictions = clf.predict(X_test)
f1_score(Y_test, Y_predictions), accuracy_score(Y_test, Y_predictions)

(271, 72123) (271,)
(68, 72123)


(0.0, 0.7058823529411765)

In [46]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error

labels = get_affect_regression

X_train = vectorizer.transform(datasets['train']['content'])
Y_train = labels(datasets['train'])*100
X_test = vectorizer.transform(datasets['test']['content'])
Y_test = labels(datasets['test'])*100

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)
clf = GaussianNB()
Y_1 =  (Y_train[:,0])
clf.fit(X_train.toarray(), Y_1.astype(int))
Y_pred = clf.predict(X_test.toarray())

clf1 = GaussianNB()
clf1.fit(X_train.toarray(), Y_train[:,1].astype(int))
Y_pred1 = clf1.predict(X_test.toarray())

Y_pred = np.stack([Y_pred, Y_pred1], axis=-1)

mean_squared_error(Y_test, Y_pred, squared=False)/10


(271, 72123) (271, 2)
(68, 72123) (68, 2)


1.1285076037691133

In [53]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

labels = get_agreement
X_train = vectorizer.transform(datasets['train']['content'])
Y_train = labels(datasets['train'])*100
X_test = vectorizer.transform(datasets['test']['content'])
Y_test = labels(datasets['test'])*100


clf = GaussianNB()
clf.fit(X_train.toarray(), Y_train.astype(int))
Y_pred = clf.predict(X_test.toarray())


mean_squared_error(Y_test, Y_pred, squared=False)/10


3.937266468185301

In [19]:
print(Y_pred)

[0.17360143 0.1363708  0.13508032 0.13267839 0.13584559 0.12462874
 0.118868   0.12263689 0.12833155 0.12323239 0.11928421 0.12911806
 0.12246143 0.14045255 0.12066204 0.15137224 0.15664313 0.12131465
 0.13111263 0.14161968 0.13032109 0.11825588 0.1502283  0.12545985
 0.18479262 0.13273704 0.12721439 0.10367145 0.14953232 0.1153099
 0.16138302 0.14329987 0.11965157 0.16355637 0.12675128 0.14537805
 0.15595305 0.13546984 0.16193436 0.0943947  0.14065102 0.12202631
 0.1401813  0.1404608  0.15070951 0.13554872 0.1027411  0.13820544
 0.12917506 0.12272706 0.13756466 0.13305628 0.14185996 0.1598903
 0.11505355 0.14452019 0.11255087 0.14788757 0.11744001 0.13168944
 0.13862449 0.1329071  0.09533359 0.12495422 0.10957039 0.18235641
 0.14846088 0.1456744 ]
